In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

import re
from bs4 import BeautifulSoup
from tqdm import tqdm
from nltk.stem import WordNetLemmatizer
import nltk

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score, confusion_matrix, recall_score, precision_score

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense , Input , LSTM , Embedding, Dropout , Activation, GRU, Flatten
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model, Sequential
from keras.layers import Convolution1D
from keras import initializers, regularizers, constraints, optimizers, layers

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
Using TensorFlow backend.


In [ ]:
save_path = '/content/drive/My Drive/Natural Language Processing/Reviews.csv'
df = pd.read_csv(save_path)

In [ ]:
df["Label"] = df["Score"].apply(lambda x : 1 if x>=3 else 0)

In [ ]:
final_df=df.drop_duplicates(subset={"Score","UserId","ProfileName","Time","Text"}, keep='first', inplace=False)
final_df.shape

(394008, 11)

In [ ]:
def decontract(text):
    text = re.sub(r"won\'t", "will not", text)
    text = re.sub(r"can\'t", "can not", text)
    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'m", " am", text)
    return text



In [ ]:
#set of custom stop words
stop_words= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

In [ ]:
import nltk
lemmatizer = WordNetLemmatizer()
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
def preprocess_text(review):
    review = re.sub(r"http\S+", "", review)             # removing website links
    review = BeautifulSoup(review, 'lxml').get_text()   # removing html tags
    review = decontract(review)                         # decontracting
    review = re.sub("\S*\d\S*", "", review).strip()     # removing the words with numeric digits
    review = re.sub('[^A-Za-z]+', ' ', review)          # removing non-word characters
    review = review.lower()                             # converting to lower case
    review = [word for word in review.split(" ") if not word in stop_words] # removing stop words
    review = [lemmatizer.lemmatize(token, "v") for token in review] #Lemmatization
    review = " ".join(review)
    review.strip()
    return review
final_df['Text'] = final_df['Text'].apply(lambda x: preprocess_text(x))

In [ ]:
final_df['Text'] 

0         I have bought several of the Vitality canned d...
1         Product arrived labeled as Jumbo Salted Peanut...
2         This is a confection that has been around a fe...
3         If you are looking for the secret ingredient i...
4         Great taffy at a great price.  There was a wid...
                                ...                        
568449    Great for sesame chicken..this is a good if no...
568450    I'm disappointed with the flavor. The chocolat...
568451    These stars are small, so you can give 10-15 o...
568452    These are the BEST treats for training and rew...
568453    I am very satisfied ,product is as advertised,...
Name: Text, Length: 394008, dtype: object

In [ ]:
train_df, test_df = train_test_split(final_df, test_size = 0.1, random_state = 42)
print("Training data size : ", train_df.shape)
print("Test data size : ", test_df.shape)

Training data size :  (354607, 11)
Test data size :  (39401, 11)


In [ ]:
train_text = train_df["Text"]
train_score = train_df["Label"]

test_text = test_df["Text"]
test_score = test_df["Label"]

In [ ]:
df_positive = train_df[train_df["Label"]==1]
df_negative = train_df[train_df["Label"]==0]

In [ ]:
df_positive_text = df_positive["Text"]
df_negative_text = df_negative["Text"]

In [ ]:
df_negative_up = df_negative_text.sample(len(df_positive_text), replace=True)
df_positive_up = df_positive_text
df_positive_down = df_positive_text.sample(len(df_negative_text))
df_negative_down = df_negative_text

In [ ]:
n=2
df_negative_big = df_negative_text.sample(n*len(df_negative_text), replace = True)
df_positive_big = df_positive_text.sample(n*len(df_positive_text), replace = True)

In [ ]:
X_big = df_positive_big.append(df_negative_big)
y_big = np.zeros((len(X_big),1))
l = len(df_positive_big)
y_big[0:l] = 1

In [ ]:
print(len(df_negative_up))
print(len(df_positive_up))
print(len(df_negative_down))
print(len(df_positive_down))
print(len(df_negative_big))
print(len(df_positive_big))

303128
303128
51479
51479
102958
606256


In [ ]:
X_down = df_positive_down.append(df_negative_down)
X_up = df_positive_up.append(df_negative_up)
y_down = np.zeros((len(df_negative_down)+len(df_positive_down),1))
y_down[0:len(df_positive_down)] = 1
y_up = np.zeros((len(df_negative_up)+len(df_positive_up),1))
y_up[0:len(df_positive_up)] = 1
print(y_down)
print(y_up)

[[1.]
 [1.]
 [1.]
 ...
 [0.]
 [0.]
 [0.]]
[[1.]
 [1.]
 [1.]
 ...
 [0.]
 [0.]
 [0.]]


In [ ]:
top_words = 10000
tokenizer = Tokenizer(num_words=top_words)
tokenizer.fit_on_texts(train_df['Text'])

In [ ]:
X_up

257779    I am very happy with this wine in a box kit. S...
81168     For some reason, they sent me the "mini" small...
361298    <a href="http://www.amazon.com/gp/product/B003...
22075     The toy got a lot of interest, but I eventuall...
119395    I'm so glad to see this on Amazon!  I first bo...
                                ...                        
99491     It does not taste bad, but I thought I was pur...
418958    I totally agreed with all of you who bought th...
501061    I bought this product three times. The first t...
315952    They show you a picture of the good opaque whi...
136101    Many people seem to love this tea, but it wasn...
Name: Text, Length: 606256, dtype: object

In [ ]:
list_tokenized_train = tokenizer.texts_to_sequences(train_df['Text'])
max_review_length = 50
X_train = pad_sequences(list_tokenized_train, maxlen=max_review_length)
y_train = train_df['Label']

In [ ]:
list_tokenized_train[2]

[4,
 408,
 395,
 391,
 62,
 278,
 403,
 34,
 430,
 2484,
 103,
 123,
 434,
 1243,
 104,
 431,
 565,
 196,
 7,
 761,
 4,
 2,
 55,
 1,
 430,
 2484,
 103,
 123,
 2739,
 23,
 242,
 96,
 56,
 1,
 60,
 1621,
 31,
 3564,
 8660,
 2,
 17,
 92,
 23,
 21,
 1466,
 11,
 33,
 3,
 1264,
 3,
 1855,
 2,
 92,
 4,
 318,
 3267,
 2146,
 5,
 414,
 16,
 902,
 3,
 1731,
 7,
 434,
 2,
 150,
 40,
 1,
 208,
 474,
 1327,
 3,
 58,
 1611,
 2,
 426,
 1,
 2739,
 5,
 1,
 521,
 12,
 2117,
 33,
 3,
 1264,
 23,
 21,
 99,
 212,
 5,
 87,
 11,
 13,
 798,
 11,
 1,
 283,
 9,
 8,
 4,
 39,
 580,
 32,
 4,
 5737,
 83]

In [ ]:
list_tokenized_train_up = tokenizer.texts_to_sequences(X_up)
max_review_length = 50
X_train_up = pad_sequences(list_tokenized_train_up, maxlen=max_review_length)
y_train_up = y_up

In [ ]:
embedding_vetor_length = 32
model = Sequential()
model.add(Embedding(top_words+1, embedding_vetor_length, input_length=max_review_length))
model.add(LSTM(50, return_sequences=True))
model.add(LSTM(50, return_sequences=True))
model.add(LSTM(50))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 50, 32)            320032    
_________________________________________________________________
lstm_7 (LSTM)                (None, 50, 50)            16600     
_________________________________________________________________
lstm_8 (LSTM)                (None, 50, 50)            20200     
_________________________________________________________________
lstm_9 (LSTM)                (None, 50)                20200     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 51        
Total params: 377,083
Trainable params: 377,083
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.load_weights('/content/drive/My Drive/Natural Language Processing/coef.h5')

In [ ]:
model.fit(X_train,y_train, nb_epoch=10, batch_size=512, validation_split=0.1)
model.save('/content/drive/My Drive/Natural Language Processing/coef.h5')

Train on 319146 samples, validate on 35461 samples
Epoch 1/10
319146/319146 [==============================] - 440s 1ms/step - loss: 0.1316 - accuracy: 0.9459 - val_loss: 0.2130 - val_accuracy: 0.9173
Epoch 2/10
319146/319146 [==============================] - 442s 1ms/step - loss: 0.1205 - accuracy: 0.9506 - val_loss: 0.2217 - val_accuracy: 0.9137
Epoch 3/10
319146/319146 [==============================] - 447s 1ms/step - loss: 0.1097 - accuracy: 0.9556 - val_loss: 0.2486 - val_accuracy: 0.9139
Epoch 4/10
319146/319146 [==============================] - 454s 1ms/step - loss: 0.0997 - accuracy: 0.9604 - val_loss: 0.2522 - val_accuracy: 0.9113
Epoch 5/10
319146/319146 [==============================] - 458s 1ms/step - loss: 0.0907 - accuracy: 0.9642 - val_loss: 0.2811 - val_accuracy: 0.9112
Epoch 6/10
 98304/319146 [========>.....................] - ETA: 5:09 - loss: 0.0749 - accuracy: 0.9710

KeyboardInterrupt: ignored

In [ ]:
model.save('/content/drive/My Drive/Natural Language Processing/coef.h5')

In [ ]:
model.load_weights('/content/drive/My Drive/Natural Language Processing/coef_up.h5')

In [ ]:
model.fit(X_train_up,y_train_up, nb_epoch=8, batch_size=512, validation_split=0.1)
model.save('/content/drive/My Drive/Natural Language Processing/coef_up.h5')

Train on 545630 samples, validate on 60626 samples
Epoch 1/8
545630/545630 [==============================] - 762s 1ms/step - loss: 0.1301 - accuracy: 0.9547 - val_loss: 0.1396 - val_accuracy: 0.9592
Epoch 2/8
545630/545630 [==============================] - 737s 1ms/step - loss: 0.0991 - accuracy: 0.9667 - val_loss: 0.1209 - val_accuracy: 0.9673
Epoch 3/8
545630/545630 [==============================] - 735s 1ms/step - loss: 0.0809 - accuracy: 0.9736 - val_loss: 0.0983 - val_accuracy: 0.9723
Epoch 4/8
545630/545630 [==============================] - 731s 1ms/step - loss: 0.0683 - accuracy: 0.9785 - val_loss: 0.0750 - val_accuracy: 0.9835
Epoch 5/8
545630/545630 [==============================] - 768s 1ms/step - loss: 0.0585 - accuracy: 0.9821 - val_loss: 0.0663 - val_accuracy: 0.9849
Epoch 6/8
534016/545630 [============================>.] - ETA: 16s - loss: 0.0516 - accuracy: 0.9845

KeyboardInterrupt: ignored

In [ ]:
list_tokenized_test = tokenizer.texts_to_sequences(test_df['Text'])
X_test = pad_sequences(list_tokenized_test, maxlen=max_review_length)
y_test = test_df['Label']
prediction = model.predict(X_test)
y_pred = (prediction > 0.5)
print("Accuracy of the model : ", accuracy_score(y_pred, y_test))
print('F1-score: ', f1_score(y_test, y_pred, pos_label=0))
recall_score_value = recall_score(y_test, y_pred, pos_label=0)
precision_score_value = precision_score(y_test, y_pred, pos_label=0)
print('Recall: {}'.format(recall_score_value))
print('Precision: {}'.format(precision_score_value))
print('Confusion matrix: {}'.format(confusion_matrix(y_test,y_pred)))

Accuracy of the model :  0.9002817187381031
F1-score:  0.6655316250957691
Recall: 0.6930851063829787
Precision: 0.6400851481906009
Confusion matrix: [[ 3909  1731]
 [ 2198 31563]]


In [ ]:
# Test for model upsampling
list_tokenized_test = tokenizer.texts_to_sequences(test_df['Text'])
X_test = pad_sequences(list_tokenized_test, maxlen=max_review_length)
y_test = test_df['Label']
prediction = model.predict(X_test)
y_pred = (prediction > 0.5)
print("Accuracy of the model : ", accuracy_score(y_pred, y_test))
print('F1-score: ', f1_score(y_test, y_pred, pos_label=0))
recall_score_value = recall_score(y_test, y_pred, pos_label=0)
precision_score_value = precision_score(y_test, y_pred, pos_label=0)
print('Recall: {}'.format(recall_score_value))
print('Precision: {}'.format(precision_score_value))
print('Confusion matrix: {}'.format(confusion_matrix(y_test,y_pred)))

Accuracy of the model :  0.9008654602675059
F1-score:  0.6637975555173007
Recall: 0.6836879432624113
Precision: 0.6450317832050854
Confusion matrix: [[ 3856  1784]
 [ 2122 31639]]


In [ ]:
embedding_vecor_length = 32
model_1 = Sequential()
model_1.add(Embedding(top_words+1, embedding_vecor_length, input_length=max_review_length))
model_1.add(LSTM(100, return_sequences=True))
model_1.add(LSTM(100))
#model_1.add(Dense(50, activation='sigmoid'))
model_1.add(Dense(1, activation='sigmoid'))
model_1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_1.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 130, 32)           192032    
_________________________________________________________________
lstm_5 (LSTM)                (None, 130, 100)          53200     
_________________________________________________________________
lstm_6 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 101       
Total params: 325,733
Trainable params: 325,733
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.load_weights('/content/drive/My Drive/Natural Language Processing/coef_1.h5')

In [ ]:
model_1.fit(X_train,y_train, nb_epoch=5, batch_size=2048, validation_split=0.05)
model_1.save('/content/drive/My Drive/Natural Language Processing/coef_1.h5')

Train on 336876 samples, validate on 17731 samples
Epoch 1/5
336876/336876 [==============================] - 1585s 5ms/step - loss: 0.2039 - accuracy: 0.9158 - val_loss: 0.2112 - val_accuracy: 0.9138
Epoch 2/5
336876/336876 [==============================] - 1655s 5ms/step - loss: 0.1959 - accuracy: 0.9191 - val_loss: 0.2059 - val_accuracy: 0.9165
Epoch 3/5
336876/336876 [==============================] - 1639s 5ms/step - loss: 0.1883 - accuracy: 0.9218 - val_loss: 0.2052 - val_accuracy: 0.9160
Epoch 4/5
336876/336876 [==============================] - 1637s 5ms/step - loss: 0.1817 - accuracy: 0.9249 - val_loss: 0.2014 - val_accuracy: 0.9153
Epoch 5/5
336876/336876 [==============================] - 1642s 5ms/step - loss: 0.1742 - accuracy: 0.9277 - val_loss: 0.1990 - val_accuracy: 0.9168


In [ ]:
#list_tokenized_test = tokenizer.texts_to_sequences(test_df['Text'])
#X_test = pad_sequences(list_tokenized_test, maxlen=max_review_length)
#y_test = test_df['Label']
prediction = model_1.predict(X_test)
y_pred = (prediction > 0.5)
print("Accuracy of the model : ", accuracy_score(y_pred, y_test))
print('F1-score: ', f1_score(y_test, y_pred, pos_label=0))
recall_score_value = recall_score(y_test, y_pred, pos_label=0)
precision_score_value = precision_score(y_test, y_pred, pos_label=0)
print('Recall: {}'.format(recall_score_value))
print('Precision: {}'.format(precision_score_value))
print('Confusion matrix: {}'.format(confusion_matrix(y_test,y_pred)))

Accuracy of the model :  0.9186568868810436
F1-score:  0.6818226943313809
Recall: 0.6088652482269503
Precision: 0.774644710128581
Confusion matrix: [[ 3434  2206]
 [  999 32762]]
